In [31]:
import numpy as np
import imageio
import cv2
import matplotlib.pyplot as plt

im = imageio.imread('photo2.jpg')


In [16]:
# 获取RGB图像
def GetRGB(path):
    im_BGR = cv2.imread(path,cv2.COLOR_GRAY2RGB)
    im = cv2.cvtColor(im_BGR,cv2.COLOR_BGR2RGB)
    return im


In [1]:
# 展示直方图
def DrawHist(input_img,pic_handle,histogram_handle):
    kwargs = dict(bins = 50, histtype='bar', edgecolor = "white",alpha=0.5, density  = True)
    pic_handle.set_title("The image")
    pic_handle.imshow(input_img)
    histogram_handle.set_title("The histogram")
    histogram_handle.hist(input_img.flatten(),**kwargs)

imtest0 = imageio.imread('imageio:chelsea.png')
imtest1 = GetRGB('./3_3Photo/3.jpg')

plt.figure(figsize=(20,10))
DrawHist(imtest0,plt.subplot(221),plt.subplot(222))
DrawHist(imtest1,plt.subplot(223),plt.subplot(224))

plt.show()

NameError: name 'imageio' is not defined

In [2]:
# 尝试直方图均衡化， 
def CaculateHistogram(input_image):
    # 参数1. 单通道8位灰度图像
    # 输出1. 总像素值  
    # 输出2. 灰度值分布计数
    # 输出3. 灰度值分布前缀和（用来做后续直方图平局奴化）
    
    # 区分通道
    if len(np.shape(input_image)) == 3:
           height,width,level = np.shape(input_image)
           summ = height*width*level
    else :
           height,width = np.shape(input_image)
           summ = height*width

    
#     第一种 通过访问指定位置像素  耗时executed in 7m 9s
#     巨慢无比
#     for i in range(height):
#         for j in range(width):
#             caculate_num[input_image[i][j]] += 1
            
#     第二种 直接拆分行，像素获取像素值  耗时executed in 14.4s
#     for line in input_image:
#         for px in line:
#             caculate_num[px] +=1 

#     第三种 调用np.histogram()方法  耗时executed in 644ms
    caculate_num,index_x = GetHistogramArray(input_image)
    caculate_num = np.append(caculate_num,1)
#     print(np.shape(caculate_num))
    #     第四种 吕少推荐的方法(还没看懂)
#     caculate_num = cumsum(input_image,256)
#     print(np.shape(caculate_num))
    
    sum_num = np.copy(caculate_num)
    for i in range(1,256):
#         print(sum_num[i-1],caculate_num[i])
        sum_num[i] = sum_num[i-1] + sum_num[i]
    return summ,caculate_num,sum_num
    
def GetHistogramArray(image):
    return np.histogram(image,np.arange(0,256));

def cumsum(img, bins):
    
    histogram = np.zeros(bins)
    for pixel in np.arange(0, bins, 1):
        histogram[pixel] += len(img[img==pixel])
        
    return histogram
    
def HistogramEqualizationLUT(input_image):
    # 参数1, 单通道8位灰度图像
    # 输出1. 单通道8位灰度图像（直方图均衡化后）
    size,data,data_sum = CaculateHistogram(input_image)
    fxy  = lambda x: (255*data_sum[x])//size
    table = np.array([fxy(i) for i in range(256)])
#     print(table)
#     不可以这样
#     image_new = table[input_image]
    lut = lambda x: table[x]
    return lut(input_image),table


imtest0_he,imtest0_tabel = HistogramEqualizationLUT(imtest0)
imtest1_he,imtest1_tabel = HistogramEqualizationLUT(imtest1)
plt.figure(figsize=(20,20))
DrawHist(imtest0,plt.subplot(421),plt.subplot(422))
DrawHist(imtest0_he,plt.subplot(423),plt.subplot(424))
DrawHist(imtest1,plt.subplot(425),plt.subplot(426))
DrawHist(imtest1_he,plt.subplot(427),plt.subplot(428))

NameError: name 'imtest0' is not defined

In [3]:
plt.figure(figsize=(15,5))
plotline = plt.subplot(121)
plotline.set_title("The function")
plotline.plot(np.arange(256),np.arange(256),color = 'black',label = "$r$")
plotline.plot(np.arange(256),imtest0_tabel,label = "$T(r)$")
plotline.plot(np.arange(256),imtest1_tabel,label = "$G(z)$")
plotline.legend()

kwargs = dict(bins = 50, histtype='bar', edgecolor = "white",alpha=0.6, density  = True)
histogram = plt.subplot(122)
histogram.set_title("The histogram")
histogram.hist(imtest0.flatten(),**kwargs, label = "imtest0 $p_r(r)$" )
histogram.hist(imtest1.flatten(), **kwargs, label = "imtest1 $p_z(z)$")
histogram.legend()
plt.show()

NameError: name 'plt' is not defined

In [4]:
# 获取目标图像累积分布函数的反函数lut表（离散）
def GetReverseTable(raw_table):

    new_table = np.zeros(256,dtype = np.int64)
    cnt = 0
    for i in range(256):
        while cnt<=raw_table[i]:
            new_table[cnt] = i
            cnt+=1
    return new_table

reverse_imtest1_table = GetReverseTable(imtest1_tabel)
match_imtest0_table = reverse_imtest1_table[imtest0_tabel]
match_imtest0 = match_imtest0_table[imtest0]



NameError: name 'imtest1_tabel' is not defined

In [5]:
# 加入反函数，再绘图
plt.figure(figsize=(15,5))
plotline = plt.subplot(121)
plotline.set_title("The function")
plotline.plot(np.arange(256),imtest0_tabel,label = "$T(r)$")
plotline.plot(np.arange(256),np.arange(256),color = "black",label = "$r$")
plotline.plot(np.arange(256),imtest1_tabel,label = "$G(z)$")
plotline.plot(np.arange(256),reverse_imtest1_table, label = "$G^{-1}(z)$")
plotline.plot(np.arange(256),match_imtest0_table, label= "$G^{-1}(T(r)), N(r)$")
plotline.legend()


# 设置直方图参数 分割块 叠加模式 边界颜色 透明度 正则化（映射到0~1）
kwargs = dict(bins = 50, histtype='bar', edgecolor = "white",alpha=0.5, density  = True)
histogram1 = plt.subplot(122)
histogram1.set_title("The histogram")
histogram1.hist(imtest0.flatten(),color = "black",label = "imtest0 original",**kwargs )
histogram1.hist(imtest1.flatten(),label = "imtest1 original",**kwargs)
histogram1.hist(match_imtest0.flatten(),label = "imtest0 after match",**kwargs)
histogram1.legend()

plt.show()

NameError: name 'plt' is not defined

In [6]:
# 输出三种图像
plt.figure(figsize=(20,15))
DrawHist(imtest0,plt.subplot(321),plt.subplot(322))
DrawHist(imtest0_he,plt.subplot(323),plt.subplot(324))
DrawHist(match_imtest0,plt.subplot(325),plt.subplot(326))
plt.show()

NameError: name 'plt' is not defined